## Imports

In [4]:
import os
from dotenv import load_dotenv
import logging

from azure.ai.projects.aio import AIProjectClient
from azure.ai.agents.models import (
    Agent,
    AsyncToolSet,
    AzureAISearchTool,
    FilePurpose,
    FileSearchTool,
    Tool,
)
from azure.ai.projects.models import ConnectionType, ApiKeyCredentials
from azure.identity.aio import DefaultAzureCredential
from azure.core.credentials_async import AsyncTokenCredential

load_dotenv()

True

In [ ]:
agent = None

proj_endpoint = os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT")
agent_id = os.environ.get("AZURE_AI_AGENT_ID")

async def create_agent(ai_client: AIProjectClient, creds: AsyncTokenCredential) -> Agent:
    logging.info("Creating new agent with resources")
    # tool = await get_available_tool(ai_client, creds)
    toolset = AsyncToolSet()
    # toolset.add(tool)
    
    instructions = "You are a helpful assistant. Use the tools provided to answer questions."
    
    agent = await ai_client.agents.create_agent(
        model=os.environ["AZURE_AI_AGENT_DEPLOYMENT_NAME"],
        name=os.environ["AZURE_AI_AGENT_NAME"],
        instructions=instructions,
        toolset=toolset
    )
    return agent

async def initialize_resources():
    try: 
        async with DefaultAzureCredential(exclude_shared_token_cache_credential=True) as creds:
            async with AIProjectClient(credential=creds, endpoint=proj_endpoint) as ai_client: # type: ignore
                if agent_id is not None: 
                    try:
                        agent = await ai_client.agents.get_agent(agent_id)
                        logging.info(f"Found agent by ID: {agent.id}")
                        return
                    except Exception as e:
                        logging.warning(f"Could not retrieve agent by AZURE_AGENT_ID = \"{agent_id}\". Will create a new agent.")
                
                agent_list = ai_client.agents.list_agents()
                if agent_list:
                    async for agent_object in agent_list:
                        if agent_object.name == os.environ["AZURE_AI_AGENT_NAME"]:
                            logging.info(
                                f"Found existing agent named '{agent_object.name}', ID: {agent_object.id}")
                            os.environ["AZURE_AI_AGENT_ID"] = agent_object.id
                            return
                
                # Create a new agent
                agent = await create_agent(ai_client, creds)
                os.environ["AZURE_AI_AGENT_ID"] = agent.id
                logging.info(f"Created agent, agent ID: {agent.id}")
    except Exception as e:
        logging.info("Error creating agent: {e}", exc_info=True)
        raise RuntimeError(f"Failed to create the agent: {e}")
    

await initialize_resources()